# Introduction + Set-up

TensorFlow is a powerful tool to develop any machine learning pipeline. This model explores the concept of using neural style transfer to augment medical images.

Medical images are rarely ever processed in the same way. Different institutions, different machines, different technicians, and many other factors lead to variation within medical scans. These variations can impact how well an ML model can learn from given data. If images processed by institution #1 work better with the model than images processed by institution #2, we would want our institution #2 images to look more like institution #1 images.

Neural style transfer is outlined in ["A Neural Algorithm of Artistic Style"](https://arxiv.org/abs/1508.06576) (Gatys et al.). On a very high level, neural style transfer is used to compose on image in the style of another using deep learning. This notebook shows that neural style transfer does indeed improve scores for images processed by lower-scoring methods.

In [1]:
! pip install tensorflow==2.2.0 -q

ERROR: Could not find a version that satisfies the requirement tensorflow==2.2.0 (from versions: 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0)
ERROR: No matching distribution found for tensorflow==2.2.0


In [2]:
import os
import PIL
import math
import warnings
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from kaggle_datasets import KaggleDatasets
from sklearn.model_selection import train_test_split

SEED = 1337
print('Tensorflow version : {}'.format(tf.__version__))

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.get_strategy() # for CPU and single GPU
    
print('Number of replicas:', strategy.num_replicas_in_sync)

ModuleNotFoundError: No module named 'kaggle_datasets'

# Data loading

The first step is to load in our data. We will be working with the PANDA data that was used for the Prostate Cancer Grade Assessment Challenge. The tiling for this dataset is explained in my first [PANDA notebook](https://www.kaggle.com/amyjang/tensorflow-cnn-data-augmentation-prostate-cancer). The tiles have been aggregated into singular images for your convenience in a new [dataset](https://www.kaggle.com/amyjang/pandatilesagg).

The PANDA data contains prostate cancer microscopy scans from two different institutions - Radboud University Medical Center and Karolinska Institute. We'll run our ML classifying model on the two sets of images separately to see which institution has better performing data.

In [ ]:
MAIN_DIR = '../input/pandatilesagg'
TRAIN_IMG_DIR = os.path.join(MAIN_DIR, 'all_images')
train_csv = pd.read_csv(os.path.join(MAIN_DIR, 'train.csv'))

In [ ]:
radboud_csv = train_csv[train_csv['data_provider'] == 'radboud']
karolinska_csv = train_csv[train_csv['data_provider'] != 'radboud']
img_ids = train_csv['image_id']

In [ ]:
r_train, r_test = train_test_split(
    radboud_csv,
    test_size=0.2, random_state=SEED
)

k_train, k_test = train_test_split(
    karolinska_csv,
    test_size=0.2, random_state=SEED
)

Generally, it is better practice to specify constant variables than it is to hard-code numbers. This way, changing parameters is more efficient and complete. Specfiy some constants below.

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
IMG_DIM = (1536, 128)
CLASSES_NUM = 6
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
EPOCHS = 100
N=12

The first step is to create a function that will decode our image into a tensor. We can use the `tf.image` API.

In [ ]:
def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  return tf.image.resize(img, IMG_DIM)

This second function maps the filepath to the the image and label. The files are labeled so that the label - the ISUP grade - is the first part of the file name and the image id is the second part.

In [ ]:
def get_item(file_path):    
    image = tf.io.read_file(file_path)
    image = decode_img(image)
    label = tf.strings.split(file_path, '_')
    label = tf.strings.to_number(label[-2])
    label = tf.cast(label, tf.int32)
    
    return image, tf.one_hot(label, CLASSES_NUM)

In [ ]:
r_train['isup_grade'] = r_train['isup_grade'].apply(str)
r_train['file'] = TRAIN_IMG_DIR +'/_' + r_train['isup_grade'] + '_' + r_train['image_id'] + '.jpg'

r_test['isup_grade'] = r_test['isup_grade'].apply(str)
r_test['file'] = TRAIN_IMG_DIR +'/_' + r_test['isup_grade'] + '_' + r_test['image_id'] + '.jpg'

k_train['isup_grade'] = k_train['isup_grade'].apply(str)
k_train['file'] = TRAIN_IMG_DIR +'/_' + k_train['isup_grade'] + '_' + k_train['image_id'] + '.jpg'

k_test['isup_grade'] = k_test['isup_grade'].apply(str)
k_test['file'] = TRAIN_IMG_DIR +'/_' + k_test['isup_grade'] + '_' + k_test['image_id'] + '.jpg'

In [ ]:
def get_dataset(df):
    ds = tf.data.Dataset.from_tensor_slices(df['file'].values)
    ds = ds.map(get_item, num_parallel_calls=AUTOTUNE)
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

We're going to define two different datasets: one with radboud images and one with the karolinska images.

In [ ]:
r_train_ds = get_dataset(r_train)
k_train_ds = get_dataset(k_train)

# Visualize our input data

Run the following cell to define the method to visualize our input data. This method displays the new images and their corresponding ISUP grade.

In [ ]:
def show_batch(image_batch, label_batch):
    plt.figure(figsize=(10,10))
    for n in range(10):
        ax = plt.subplot(1,10,n+1)
        plt.imshow(image_batch[n])
        plt.axis("off")

**Radboud Scans**

In [ ]:
image_batch, label_batch = next(iter(r_train_ds))
show_batch(image_batch, label_batch)

**Karolinska Scans**

In [ ]:
image_batch, label_batch = next(iter(k_train_ds))
show_batch(image_batch, label_batch)

We see from these two visualizations that the Radboud and Karolinska images look very different. Even from a quick initial glance, we se that the hues and colors are different. Will the variations affect how well the images do in our model?

# Build our classifying model

We will be utilizing the VGG16 pre-trained model to classify our data. Since the base model has already been trained with imagenet weights, we do not want the weights to change, so the base mode must not be trainable. However, the number of classes that our model has differs from the original model. Therefore, we do not want to include the top layers because we will add our own Dense layer that has the same number of nodes as our output class.

In [ ]:
def make_model():
    base_model = tf.keras.applications.VGG16(input_shape=(*IMG_DIM, 3),
                                             include_top=False,
                                             weights='imagenet')
    
    base_model.trainable = True
    
    model = tf.keras.Sequential([
        base_model,
        
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(CLASSES_NUM, activation='softmax'),
    ])
    
    model.compile(optimizer=tf.keras.optimizers.RMSprop(),
                  loss='categorical_crossentropy',
                  metrics=[tf.keras.metrics.AUC(name='auc')])
    
    return model

Learning rate is a very important hyperparameter, and it can be difficult to choose the "right" one. A learning rate that it too high will prevent the model from converging, but one that is too low will be far too slow. We will utilize multiple callbacks, using the `tf.keras` API to make sure that we are using an ideal learning rate and to prevent the model from overfitting. We can also save our model so that we do not have to retrain it next time.

In [ ]:
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 **(epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(0.01, 20)

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10,
                                                     restore_best_weights=True)

# Compare scores between institutions

Let's run our model on the Radboud images first.

In [ ]:
with strategy.scope():
    model = make_model()

history = model.fit(
    r_train_ds, epochs=20,
    callbacks=[early_stopping_cb, lr_scheduler]
)

The ROC AUC score for the radboud images are quite low, near 0.5. This means that the the model finds it difficult to distinguish between the images of the different class.

Let's run the same model on the Karolinska images.

In [ ]:
with strategy.scope():
    model = make_model()
    
history = model.fit(
    k_train_ds, epochs=20,
    callbacks=[early_stopping_cb, lr_scheduler]
)

The ROC AUC scores are way higher for the karolinska images. This can be a problem because the discrepancy shows that the images from the different institutions do in fact have noticable variances by the machine. It can also cause the model to be biased towards karolinska images, and this can hurt patients with radboud images.

The idea is that we will use neural style transfer to change radboud images to look more like karolinska images.

# Neural style transfer intro

Let's first visualize a radboud image and a karolinska image next to each other.

A more in-depth tutorial of neural style transfer for artistic painting transformations can be found [here](https://www.tensorflow.org/tutorials/generative/style_transfer).

In [ ]:
image_batch, _ = next(iter(r_train_ds))
r_image = image_batch[4].numpy()
plt.figure(figsize=(10,10))
ax = plt.subplot(1,2,1)
plt.title("radboud")
plt.imshow(r_image)
plt.axis("off")

image_batch, _ = next(iter(k_train_ds))
k_image = image_batch[4].numpy()
ax = plt.subplot(1,2,2)
plt.imshow(k_image)
plt.title("karolinska")
plt.axis("off")

The differences in the processing techniques is apparent.

We will be using our karolinska image as our style image because we want to change our radboud images to look more like our karolinska images. Our model takes in 4D tensors, so let's first reformat our style and content images. Before we restyle all of our radboud images, let's first visualize how the neural style transfer works on a single image.

In [ ]:
content_image = tf.expand_dims(r_image, axis = 0)
style_image = tf.expand_dims(k_image, axis = 0)

# Define style and content layers

In a deep convolution neural net, the lower layers capture lower-level features like texture and higher layers capture higher-level features such as shapes. We define the content layers and style layers below.

In [ ]:
content_layers = ['block5_conv2'] 

style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1', 
                'block4_conv1', 
                'block5_conv1']

num_content_layers = len(content_layers)
num_style_layers = len(style_layers)

# Build the neural style transfer model

Our next step is to define a model that returns a list of intermediate layer outputs for our VGG19 model.

In [ ]:
def vgg_layers(layer_names):
  """ Creates a vgg model that returns a list of intermediate output values."""
  # Load our model. Load pretrained VGG, trained on imagenet data
  vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
  vgg.trainable = False
  
  outputs = [vgg.get_layer(name).output for name in layer_names]

  model = tf.keras.Model([vgg.input], outputs)
  return model

We can extract our style features, defined by the style layers we chose earlier, and convert them to style outputs. Since our inputs are values between 0 and 255, we have to multiple the style features by 255.

In [ ]:
style_extractor = vgg_layers(style_layers)
style_outputs = style_extractor(style_image*255)

# Calculate style

Style is a rather arbitrary concept, and we want to convert it into a value that can be understood by the model. Style can be calculated by a gram matrix below.

In [ ]:
def gram_matrix(input_tensor):
  result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
  input_shape = tf.shape(input_tensor)
  num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
  return result/(num_locations)

Build a model that extracts style and content.

In [ ]:
class StyleContentModel(tf.keras.models.Model):
  def __init__(self, style_layers, content_layers):
    super(StyleContentModel, self).__init__()
    self.vgg =  vgg_layers(style_layers + content_layers)
    self.style_layers = style_layers
    self.content_layers = content_layers
    self.num_style_layers = len(style_layers)
    self.vgg.trainable = False

  def call(self, inputs):
    "Expects float input in [0,1]"
    inputs = inputs*255.0
    preprocessed_input = tf.keras.applications.vgg19.preprocess_input(inputs)
    outputs = self.vgg(preprocessed_input)
    style_outputs, content_outputs = (outputs[:self.num_style_layers], 
                                      outputs[self.num_style_layers:])

    style_outputs = [gram_matrix(style_output)
                     for style_output in style_outputs]

    content_dict = {content_name:value 
                    for content_name, value 
                    in zip(self.content_layers, content_outputs)}

    style_dict = {style_name:value
                  for style_name, value
                  in zip(self.style_layers, style_outputs)}
    
    return {'content':content_dict, 'style':style_dict}

The StyleContentModel will return the gram matrix of the style layers and content layers.

In [ ]:
extractor = StyleContentModel(style_layers, content_layers)

results = extractor(tf.constant(content_image))

The way that loss is calculated is by calculating mean square error of the model outputs to the targets, defined below. The weight of the losses (style vs. content) are defined below.

In [ ]:
style_targets = extractor(style_image)['style']
content_targets = extractor(content_image)['content']

We'll define a clipping method to keep the values between 0 and 1.

In [ ]:
def clip_0_1(image):
  return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)

opt = tf.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)

Define the weights. Higher the weight, the more important the loss will be calculated.

In [ ]:
style_weight=1e-1
content_weight=1e4

In [ ]:
def style_content_loss(outputs):
    style_outputs = outputs['style']
    content_outputs = outputs['content']
    style_loss = tf.add_n([tf.reduce_mean((style_outputs[name]-style_targets[name])**2) 
                           for name in style_outputs.keys()])
    style_loss *= style_weight / num_style_layers

    content_loss = tf.add_n([tf.reduce_mean((content_outputs[name]-content_targets[name])**2) 
                             for name in content_outputs.keys()])
    content_loss *= content_weight / num_content_layers
    loss = style_loss + content_loss
    return loss

Define the training step.

In [ ]:
def train_step(image):
  with tf.GradientTape() as tape:
    outputs = extractor(image)
    loss = style_content_loss(outputs)

  grad = tape.gradient(loss, image)
  opt.apply_gradients([(grad, image)])
  image.assign(clip_0_1(image))

# Train the neural style model

Because we aren't using a typical Keras model, we cannot run model.fit. Instead, we'll define our own training loop.

In [ ]:
steps_per_epoch = 3

def style_transfer(image):
    image = tf.expand_dims(image, axis = 0)
    image = tf.Variable(lambda : image)
    step = 0
    for m in range(steps_per_epoch):
        step += 1
        train_step(image)
        print(".", end='')
        
    return image

# Visualize the neural style transfer

Before we augment all of the images in our dataset, let's fist visualize how our augmented radboud image compared to our original one.

In [ ]:
plt.figure(figsize=(10,10))
ax = plt.subplot(1,2,1)
plt.title("radboud")
plt.imshow(r_image)
plt.axis("off")

r_aug_image = style_transfer(r_image)
ax = plt.subplot(1,2,2)
plt.imshow(r_aug_image[0])
plt.title("radboud augmented")
plt.axis("off")

We see that there a style transfer did in fact occur. Running the neural style transfer may change the original image too much, and changing the higher-level features may hurt classifiction. Therefore, we want to keep the number of epochs relatively low.

# Run neural style transfer on Radboud images

Now that we tested to see that neural style transfer works on a single radboud image, we want to run neural style on all the radboud images. Because of resource limitations, the code is commented out below. However, the output for the data augmentation is under the radboud_aug directory in the dataset.

As a note, not all the radboud images have been augmented for efficiency purposes.

In [ ]:
# ! mkdir images

In [ ]:
# for file in r_train['file'].values:
#     img = np.array(PIL.Image.open(file)) /255.0
#     img = style_transfer(img)[0] * 255.0
#     img = img.numpy()
#     im = img.astype('uint8')
#     im = PIL.Image.fromarray(im)
#     im.save("images/" + file.split('/')[-1])

In [ ]:
# import shutil
# shutil.make_archive("images", 'zip', "/kaggle/working/images")

# Load augmented images and train the model

Let's load in our augmented radboud images.

In [ ]:
def get_dataset(ds):
    ds = ds.map(get_item, num_parallel_calls=AUTOTUNE)
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

In [ ]:
rad_aug = tf.data.Dataset.list_files("../input/pandatilesagg/radboud_aug/*")
rad_aug = get_dataset(rad_aug)

Visualize the augmented radboud images.

In [ ]:
image_batch, label_batch = next(iter(rad_aug))
show_batch(image_batch, label_batch)

In [ ]:
with strategy.scope():
    model = make_model()
    
history = model.fit(
    rad_aug, epochs=20,
    callbacks=[early_stopping_cb, lr_scheduler]
)

The ROC AUC score for the augmented images score a lot higher than the original radboud score.

From 0.54 --> 0.67!! (The exact values may be different because we did not specify a random seed)

The other model does not shown signs of improvement after each epoch, but using augmented images shows visible improvements. This notebook highlights neural style transfer's potential to augment images in the medical space!!

This idea has many important implications. First, it provides a solution to the variation in processing images. It may be difficult to universalize the processing of a microscopy scan, MRI scan, or any other type of medical image, but it is a lot easier to export a software that many institutions and labs can utilize. Additionally, different scans may introduce different biases. Neural style transfer may help eliminate certain biases. And most importantly, neural style transfer can lead to higher scoring of medical images. This means better diagnoses and less error when it comes to using ML for medical images.